<a href="https://colab.research.google.com/github/rapidestlime/Eccomerce-SG-Webscraping/blob/main/amazonSG_scraping_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Import libraries

- Helps to install chromium browser properly with colab ubuntu 20.04lt


In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
# https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver and snap
apt-get update
apt-get install chromium chromium-driver
sudo apt install snapd




*   Install selenium-wire and load selenium items





In [ ]:
!pip install selenium-wire

# Load selenium components
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# load other crucial libraries
from bs4 import BeautifulSoup
import requests as req
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast
import traceback
import pprint
import gzip
import json

# 2.Chrome Driver Setup

In [ ]:
# set up chrome driver
def driversetup():
    caps = DesiredCapabilities().CHROME
    #caps["pageLoadStrategy"] = "normal"  #  Waits for full page load
    caps["pageLoadStrategy"] = "eager"  #  Waits for page to be interactive
    #caps["pageLoadStrategy"] = "none"   # Do not wait for full page load
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser with set window size
    options.add_argument("--window-size=1920,1080")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    # add user agent
    options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"')
    driver = webdriver.Chrome(desired_capabilities=caps,executable_path="/usr/bin/chromedriver",options=options)
    
    return driver

# 3.Function for initial page search driver

In [ ]:
def search_initial_driver(search_term: str):
    """
    Description:
      Set up initial driver state of the the search page based on  search term
    Arguments:
      search_term: get the product search term [str object]
    Returns:
      Driver: driver state [webdriver object]
      NumPages: returns number of pages available to scrap [int object]
    """
    # initialise base url
    url = f"https://www.amazon.sg/s?k={search_term}"
    
    
    # load search url with chrome driver 
    Driver = driversetup()
    Driver.get(url)
    time.sleep(3)
    # make sure search result element is loaded before further action
    #WebDriverWait(Driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "s-main-slot s-result-list s-search-results sg-row")))
    
    # execute smooth scrolling so that all elements are loaded successfully since they are dynamically loaded
    Driver.execute_script("window.scrollTo({top: document.body.scrollHeight, behavior: 'smooth'});")
    
    # gets number of search pages for later processing
    page_data = Driver.find_element(By.CLASS_NAME, 's-pagination-strip').text  # bs4 ver ---> data.find('span',{'class':"s-pagination-strip"})
    NumPages = None

    if page_data is None:
        print('only one page!')
    else:
        NumPages = [int(s) for s in page_data if s.isdigit()][-1]
    
    return (Driver,NumPages)

# Set search parameters

 ❗ [**Re-run from here if any issues pops up below**] ❗

In [ ]:
search_term = 'sneakers'
search = search_initial_driver(search_term)

# Initialise containers



*   intialise empty lists to store data scrapped from search pages
*   Store references for extraction from beuatifulsoup object




In [ ]:
# stores search pages data that we Selenium can grab
search_array = {'name': [],
                'image_link' : [],
                'product_link' : [],
                'page_num' : []
                }

In [ ]:
search_params = {'results': {'tag' :'div', 'id': {'class':'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20'}},
                 'name_id_key': ".find('a',{'class':'a-link-normal s-no-outline'}).find('img',{'class':'s-image'})['alt']",
                 'image_link_key': ".find('a',{'class':'a-link-normal s-no-outline'}).find('img',{'class':'s-image'})['src']",
                 'product_link_key': "e.find('a',{'class': 'a-link-normal s-no-outline'})['href']"
              }


# 4.Functions For Scraping Summary Page Info

## Create Soup Object Function

In [ ]:
def search_soup_result(Driver, result_tag, result_id):
    """
    Description:
      Grab all elements with matching tags and ids from html page source
    Arguments:
      Driver: Driver state of current search page [webdriver object]
      result_tag: html tag to search in Driver.page_source [str object]
      result_id: id of html tag to search in Driver.page_source [str object]
    Returns:
      list_elements: list of all data under specified tags found in html page [list object]
    """
    soup = BeautifulSoup(Driver.page_source, 'html.parser')
    list_elements = soup.find_all(result_tag,result_id)
    return list_elements

## Search data array Function


In [ ]:
def fill_search_array(search_array, list_elements, page_num):
    """
    Description:
      To fill up search_array container with data in list_elements
    Arguments:
      search_array: get search data placeholder variable [dictionary object]
      list_elements: get params required to locate correct data in parsed soup [dictionary object]
      page_number: current page number to scrape from [integer object]
    """
    
    list_elements = list_elements
    
    # fill data in search_array variable
    for e in list_elements:
        
        # add product name
        search_array['name'].append(e.find('a',{'class':'a-link-normal s-no-outline'}).find('img',{'class':'s-image'})['alt'])
      
        # add product image link
        search_array['image_link'].append(e.find('a',{'class':'a-link-normal s-no-outline'}).find('img',{'class':'s-image'})['src'])
        
        # add product page link
        search_array['product_link'].append('https://www.amazon.sg' + e.find('a',{'class': 'a-link-normal s-no-outline'})['href'])
    
        # add page number
        search_array['page_num'].append(page_num)

## Function to Loop Through ALL Pages of Summary Output Page

In [ ]:
def scrape_search_pages(search_array, search_params, Driver, NumPages: int):
    """
    Description:
       Loop through and produce data from all search pages based on initial search page driver(1st page)
    Arguments:
        search_array: get search data placeholder variable [dictionary object]
        search_params: get params required to locate correct data in parsed soup [dictionary object]
        Driver: get current chrome driver state [webdriver.Chrome(options=options) object]
        NumPages: get the required number of times Selenium to click for next page [integer object]
    Returns:
        search_page: Pandas Dataframe object of all search pages data
    """
    # assign Driver object
    Driver =  Driver

    # looping to get results from all pages
    for num in range(1,NumPages+1): # ensures clicking NumPages-1 times
        
        # smooth scrolling downwards so that all elements are loaded properly
        Driver.execute_script("window.scrollTo({top: document.body.scrollHeight, behavior: 'smooth'});")
        
        # get html soup and get all search results on that page
        list_elements = search_soup_result(Driver,
                                                 search_params['results']['tag'],
                                                 search_params['results']['id']
                                                 )
        
        # fill data in search_array variable
        fill_search_array(search_array,list_elements,num)
        
        # initiates scroll back top and clicking next button
        if num < NumPages: # ensures final page next button wont be clicked
            Driver.execute_script("window.scrollTo(0,0);") # scrolls back to top for smooth scrolling the next page
            next_button = Driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            next_button.click() # tells selenium to click the next button
            time.sleep(3) # gives driver enough time to load the next page, avoid scraping detection
    
    # create dataframe
    search_pages = pd.DataFrame.from_dict(search_array)
    
    Driver.close() # close off webdriver object
    return search_pages # ---> toggle no. of rows for quick testing with indexing if needed

# Run Multi Summary Pages Loop Function

 Produce dataframe from search results of "sneakers" which we derive from initial driver state "search"

In [ ]:
search_dataframe = scrape_search_pages(search_array, search_params, search[0], search[1])
search_dataframe

,name,image_link,product_link,page_num
0,[ウインブルドン] スニーカー テニス 通学靴 幅広4E WB 052,https://m.media-amazon.com/images/I/61xi0Aq-M0...,https://www.amazon.sg/Wimbledon-Sneakers-Tenni...,1
1,Adidas LIU80 Women's Grand Court TD Lifestyle ...,https://m.media-amazon.com/images/I/61nqxP+GfX...,https://www.amazon.sg/Adidas-Advantage-Sneaker...,1
2,"Puma Space Runner/ALT Running Shoes, Sneakers,...",https://m.media-amazon.com/images/I/71ZYH60WdQ...,https://www.amazon.sg/Runner-Running-Sneakers-...,1
3,ECCO Men's Street Lite Retro Sneaker,https://m.media-amazon.com/images/I/6173VPEs0f...,https://www.amazon.sg/Street-Retro-Sneaker-Bla...,1
4,[フィラ] スニーカー ディスラプター2 厚底スニーカー ボリュームソール レディース メン...,https://m.media-amazon.com/images/I/715yJi9Lf3...,https://www.amazon.sg/F0540-0001-Disruptor-Sne...,1
...,...,...,...,...
301,Lot Works LW-S7003 JSAA Type A Certified Prote...,https://m.media-amazon.com/images/I/71xIEMlXFw...,https://www.amazon.sg/LW-S7003-Certified-Prote...,7
302,"Madras Walk GORE-TEX Women's Rain Shoes, Rain ...",https://m.media-amazon.com/images/I/71cj4jwXE8...,https://www.amazon.sg/Madras-Walk-Gore-Tex-Sho...,7
303,"Midori Anzen G3590 Safety Shoes, JSAA Certifie...",https://m.media-amazon.com/images/I/81rrK8tKND...,https://www.amazon.sg/G3590-Certified-Purchase...,7
304,"Moonstar Lifestyle Black Top Leather Sneakers,...",https://m.media-amazon.com/images/I/71PGyXsiFr...,https://www.amazon.sg/Moonstar-Lifestyle-Sneak...,7


In [ ]:
#search_dataframe.to_csv('amazonsg_search_dataframe.csv')

# 5.Functions To Scrape Individual Product Pages

## Function to get PRODUCT page Soup


In [ ]:
def product_soup_result(Driver):
    """
    Description:
      Creates product page soup object for data extraction
    Arguments:
      Driver: Driver state of current search page [webdriver object]
    Returns:
      product_soup: returns beautifulsoup object of product page [beautifulsoup object]
    """
    product_soup = BeautifulSoup(Driver.page_source, 'html.parser')
    return product_soup 

## Product Data Array Function

In [ ]:
# Driver version
def fill_product_array(Driver,product_params):
    """
    Description:
      fill up product_array that stores all product data
    Arguments:
      Driver: webdriver object having state of current product url to extract datsa from [webdriver object]
      product_params: get required keys for referencing in product data [dictionary object]
    """

    # fills up arrays in product_array

    for k,v in product_params.items():
        
        if k == 'price_single':
            data = Driver.find_elements(v[0],v[1])
            if data != []:
               product_array['min_price'].append(round(float(data[0].text.replace('\n','.').strip('S$')),2))
               product_array['max_price'].append(0.00)
        
        elif k == 'price_range':
            data = Driver.find_elements(v[0],v[1])
            if data != []:
               price_range = [e.strip('S$') for e in data[0].text.split('-\n')]
               product_array['min_price'].append(round(float(price_range[0]),2))
               product_array['max_price'].append(round(float(price_range[1]),2))
        
        elif k in ('ratings','total_ratings','discount_pct'):
            data = Driver.find_elements(v[0],v[1])
            if data != []:
                if k == 'ratings':
                    product_array[k].append(round(float(data[0].text.split(' ')[0]),1))
                elif k == 'total_ratings':
                    product_array[k].append(int(data[0].text.replace(',', '').split(' ')[0]))
                elif k == 'discount_pct':
                    product_array[k].append(int(data[0].text.strip('%'))*-1)    
            else:
                product_array[k].append(0)
        
        elif k == 'x_star_ratings':
            data = [e.strip('%') for e in Driver.find_elements(v[0],v[1])[0].text.split('\n')]
            product_array['1_star_ratings'].append(int(data[9]))
            product_array['2_star_ratings'].append(int(data[7]))
            product_array['3_star_ratings'].append(int(data[5]))
            product_array['4_star_ratings'].append(int(data[3]))
            product_array['5_star_ratings'].append(int(data[1]))
        
        elif k == 'level_x_cat':
            data = Driver.find_elements(v[0],v[1])
            cat = []
            if data != []:
                cat = data[0].text.split('\n›\n')
            while len(cat) < 5:
                cat.append(None) # fill up empty sub categories, Amazon appears to give up to 5 lvls of categories
            product_array['level_1_cat'].append(cat[0])
            product_array['level_2_cat'].append(cat[1])
            product_array['level_3_cat'].append(cat[2])
            product_array['level_4_cat'].append(cat[3])
            product_array['level_5_cat'].append(cat[4])
    
        elif k in ('brand'):
            data = Driver.find_elements(v[0],v[1])
            if k == 'brand' and data != []:
                product_array[k].append(data[0].text.split(' ')[1])
            else:
                product_array[k].append(None)
        
        elif k == 'is_best_selling':
            data = Driver.find_elements(v[0],v[1])
            if data != []:
                product_array[k].append(True)
            else:
                product_array[k].append(False)
            

## Function to iterate all product pages

In [ ]:
def scrape_product_pages(search_dataframe,product_array, product_params):
    """
    Description:
      Loop through all product links and get all product data
    Arguments:
      search_dataframe: dataframe from scrape_search_pages func [pandas dataframe object]
    Returns:
      product_dataframe: dataframe containing info from product pages [pandas dataframe object]
    """
    url_list = search_dataframe
    Driver = driversetup()
    for url in url_list:
        print(url)
        Driver.get(url)
        time.sleep(1)
        Driver.execute_script("window.scrollTo({top: document.body.scrollHeight, behavior: 'smooth'});")
        #product_soup = product_soup_result(Driver)
        fill_product_array(Driver,product_params)
        #Driver.execute_script("window.scrollTo(0,0);")
    
    Driver.close()

    product_dataframe = pd.DataFrame.from_dict(product_array)

    return product_dataframe

# Initialise Product Containers

- stores all product page data

In [ ]:
product_array = {'min_price': [],
                  'max_price': [],
                  'ratings' : [], # e.g. 4.4 out of 5 or 0
                  'total_ratings': [], # e.g. " 385 global ratings" or 0
                  '1_star_ratings' : [], # use find_all e.g.  "21% of reviews have 4 stars" or 0
                  '2_star_ratings' : [],
                  '3_star_ratings' : [],
                  '4_star_ratings' : [],
                  '5_star_ratings' : [],
                  'level_1_cat': [], # use find_all e.g. exists or None
                  'level_2_cat': [],
                  'level_3_cat': [],
                  'level_4_cat': [],
                  'level_5_cat': [],
                  'is_best_selling': [], # True or False
                  'brand' : [], # exists or None
                  'discount_pct': [], # e.g. '-6%' or 0
                }

- stores keys for referencing to extract data from product soup object

In [ ]:
product_params = {'price_single': [By.CLASS_NAME, 'a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay'], # e.g. "S$79\n33"
                  'price_range': [By.CLASS_NAME, 'a-price-range'], # S$85.84-\nS$271.87
                  'ratings' : [By.CSS_SELECTOR, 'span[data-hook=rating-out-of-text]'], # e.g. 4.4 out of 5 or 0
                  'total_ratings': [By.ID, 'acrCustomerReviewText',0], # e.g. " 385 global ratings" or 0 (if have returns 2 elements just pick either)
                  'x_star_ratings' : [By.CLASS_NAME, 'a-normal.a-align-center.a-spacing-base'], # 5 star\n78%\n4 star\n10%\n3 star\n6%\n2 star\n2%\n1 star\n4%
                  'level_x_cat': [By.CLASS_NAME,'a-unordered-list.a-horizontal.a-size-small'], # Fashion\n›\nWomen\n›\nShoes\n›\nTrainers e.g. exists or None
                  'is_best_selling': [By.CLASS_NAME, 'a-icon.a-icon-addon.p13n-best-seller-badge'], # True or False
                  'brand' : [By.ID, 'bylineInfo'], # Brand: ecco , exists or None
                  'discount_pct': [By.CLASS_NAME,'a-size-large.a-color-price.savingPriceOverride.aok-align-center.reinventPriceSavingsPercentageMargin.savingsPercentage'] # e.g. '-6%' or 0
                }

# Run function to gather all product info

In [ ]:
search_dataframe = pd.read_csv('/content/drive/MyDrive/Data Files/E commerce web scraping/amazonsg_search_dataframe.csv', index_col=None)
search_dataframe = search_dataframe[search_dataframe['product_link'].str.contains('video')==False]['product_link'] # filters out unwanted amazon prime links

In [ ]:
product_dataframe = scrape_product_pages(search_dataframe[0:100],product_array,product_params) # <- test with first 100 urls
product_dataframe

https://www.amazon.sg/Wimbledon-Sneakers-Tennis-School-Shoes/dp/B08ZY5Z3ZF/ref=sr_1_1?keywords=sneakers&qid=1674616845&sr=8-1
https://www.amazon.sg/Adidas-Advantage-Sneakers-Footwear-GW2065/dp/B09VSXP5TB/ref=sr_1_2?keywords=sneakers&qid=1674616845&sr=8-2
https://www.amazon.sg/Runner-Running-Sneakers-Athletic-Exclusive/dp/B081NRW9J2/ref=sr_1_3?keywords=sneakers&qid=1674616845&sr=8-3
https://www.amazon.sg/Street-Retro-Sneaker-Black-8-8-5/dp/B099MWJ4Z6/ref=sr_1_4?keywords=sneakers&qid=1674616845&sr=8-4
https://www.amazon.sg/F0540-0001-Disruptor-Sneakers-Thick-X-Wide/dp/B07FJ3QYFF/ref=sr_1_5?keywords=sneakers&qid=1674616845&sr=8-5
https://www.amazon.sg/ECCO-Mens-Sneaker-Cognac-7-7-5/dp/B092HKTMYX/ref=sr_1_6?keywords=sneakers&qid=1674616845&sr=8-6
https://www.amazon.sg/Cloudform-Lifestyle-Sneakers-Footwear-GW9195/dp/B09N3MRF36/ref=sr_1_7?keywords=sneakers&qid=1674616845&sr=8-7
https://www.amazon.sg/Certified-Anti-static-Professional-Sneakers-19-3911tcx/dp/B007JDZU7O/ref=sr_1_8?keywords=snea

,min_price,max_price,ratings,total_ratings,1_star_ratings,2_star_ratings,3_star_ratings,4_star_ratings,5_star_ratings,level_1_cat,level_2_cat,level_3_cat,level_4_cat,level_5_cat,is_best_selling,brand,discount_pct
0,35.72,42.52,0.0,0,0,0,0,0,0,None,None,None,None,None,False,WIMBLEDON(ウインブルドン),0
1,50.43,53.76,4.0,317,8,6,15,25,47,Fashion,Women,Shoes,Trainers,None,False,adidas(アディダス),0
2,54.63,66.65,4.2,928,4,4,13,25,54,Fashion,Men,Shoes,Sports & Outdoor Shoes,Running Shoes,True,PUMA(プーマ),0
3,185.78,240.06,4.2,39,8,6,8,14,64,Fashion,Men,Shoes,None,None,False,ecco,0
4,80.26,93.95,4.4,74,6,2,6,21,65,None,None,None,None,None,False,FILA(フィラ),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,101.08,111.40,4.3,68,4,4,13,19,60,None,None,None,None,None,False,None,0
96,47.36,56.82,0.0,0,0,0,0,0,0,Fashion,Men,Shoes,Sports & Outdoor Shoes,None,False,DUNLOP,0
97,80.34,88.22,0.0,0,0,0,0,0,0,Fashion,"Uniforms, Work & Safety",Work & Utility,Men,None,False,DIADORA,0
98,95.31,111.49,4.4,62,0,3,11,32,54,Fashion,Men,Shoes,None,None,False,Chrome,0


In [ ]:
# sanity check for product_array
for k,v in product_array.items():
    print(f"{k}: {len(v)}")

min_price: 100
max_price: 100
ratings: 100
total_ratings: 100
1_star_ratings: 100
2_star_ratings: 100
3_star_ratings: 100
4_star_ratings: 100
5_star_ratings: 100
level_1_cat: 100
level_2_cat: 100
level_3_cat: 100
level_4_cat: 100
level_5_cat: 100
is_best_selling: 100
brand: 100
discount_pct: 100


In [ ]:
https://blog.devgenius.io/multi-threaded-web-scraping-with-selenium-dbcfb0635e83
https://stackoverflow.com/questions/48450594/selenium-timed-out-receiving-message-from-renderer